In [1]:
from scipy.io.wavfile import read
import numpy as np
import soundfile as sf
import sounddevice as sd
import random

audio, sr = sf.read('dog.wav', dtype='int16')  
print(sr)
sd.play(audio)

44100


In [2]:
print(audio)

[[-44 -37]
 [-45 -39]
 [-46 -43]
 ...
 [  0   1]
 [  0   2]
 [  0   0]]


In [58]:
import wave
import struct

def audio_to_binary(filename):
    with wave.open(filename, "rb") as wav_file:
        # Read the binary data from the wave file
        binary_data = wav_file.readframes(wav_file.getnframes())
        # Unpack the binary data and return it as a list of integers
        audio = list(struct.unpack("B"*len(binary_data), binary_data))
        return np.vectorize(np.binary_repr)(audio, 8)


def binary_to_audio(binary_data, filename, sample_rate=44100, sample_width=2, channels=2):
    # Open a wave file for writing
    with wave.open(filename, "w") as wav_file:
        # Set the sample width and number of channels
        wav_file.setsampwidth(sample_width)
        wav_file.setnchannels(channels)
        # Set the sample rate
        wav_file.setframerate(sample_rate)
        # Write the binary data to the wave file
        for byte in binary_data:
            data = struct.pack("B", byte)
            wav_file.writeframes(data)

In [59]:
bin_audio_list = audio_to_binary("dog.wav") #List of integer
print(bin_audio_list, len(bin_audio_list))
bin_audio_str = ''.join(bin_audio_list)
print(type(bin_audio_str))  #OK


['11010100' '11111111' '11011011' ... '00000000' '00000000' '00000000'] 286284
<class 'str'>


In [5]:
def DNA(line, l):
    Base_map = {"00":"A", "01":"G", "10":"C", "11":"T"}
    x=""
    temp = [line[i:i+2] for i in range(0, l, 2)]
    for j in range (l//2) :
         x += Base_map[temp[j]]
    
    return x

def revDNA(line, l):
    Code_map = {"A":"00", "G":"01", "C": "10", "T":"11"}
    x = ""
    temp = [line[i:i+1] for i in range(0,l)]
    for j in range(l):
        x += Code_map[temp[j]]
    return x

In [6]:
mapped_array = np.array([DNA(elem, 8) for elem in bin_audio_list])
print(mapped_array)

['TGGA' 'TTTT' 'TGCT' ... 'AAAA' 'AAAA' 'AAAA']


In [7]:
print(type(mapped_array))

<class 'numpy.ndarray'>


In [8]:
pre_encrypt = ''.join(mapped_array)


In [9]:
#print(pre_encrypt)


In [11]:
#key = np.array([random.randrange(2) for _ in range(200)])
key = 'ATATCTGCGTACGTACGTACGTACGTACGTTGAACGTAACGGTACGT'
key_size = len(key)

In [12]:
print(revDNA(key, 47))

0011001110110110011100100111001001110010011100100111001001111101000010011100001001011100100111


In [13]:
bin_key = revDNA(key, 47)

y = int(bin_key[0:17],2) ^ int(bin_audio_str[0:17],2)
print('{0:b}'.format(y))

11100111010010011


In [60]:
a = 'ATATCTGCGTACGTACGTACGTACGTACGTTGAACGTAACGGTACGTC'
la = len(a)
b = 'ATTCG'
lb = len(b)
chunk = [a[0+i:lb+i] for i in range(0, la, lb)]
print("chunk = ", chunk)


chunk =  ['ATATC', 'TGCGT', 'ACGTA', 'CGTAC', 'GTACG', 'TACGT', 'TGAAC', 'GTAAC', 'GGTAC', 'GTC']


In [43]:

def Xor_string(a,b):
    x=''
    for i in range(len(a)):
        if (a[i] == b[i]):
            x+= '0'
        else:
            x+= '1'
    return x


In [64]:
key_val = revDNA(b, lb)
temp = chunk[len(chunk)-1]
x = Xor_string(key_val[0:len(temp)*2],revDNA(temp, len(temp)))
xorval = [Xor_string(key_val, revDNA(item,lb)) for item in chunk[0:len(chunk)-1]]
xorval.append(x)

print(xorval)

['0000110111', '1110011110', '0001100101', '1010001011', '0100110000', '1111011110', '1110111011', '0100111011', '0110001011', '010001']


In [41]:
#binary_to_audio(binary_data, "example3.wav")